In [16]:
!nvidia-smi

# Install Whisper + WhisperJAV
!pip install --upgrade pip

# Install WhisperJAV WITHOUT dependencies (avoids PyAudio)
!pip install --no-deps git+https://github.com/meizhong986/WhisperJAV.git

# Install required dependencies manually
!pip install pyloudnorm soundfile pydub librosa numpy
!pip install openai-whisper ffmpeg-python pysrt srt stable-ts faster-whisper[cuda]
!pip install auditok --no-deps


# Ensure ffmpeg is available
!apt-get update
!apt-get install -y ffmpeg

Mon Jan 12 02:11:26 2026       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   39C    P8              9W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [7]:
import whisperjav
print("WhisperJAV version:", whisperjav.__version__)

2026-01-12 01:54:47 - whisperjav - WARNING - Auditok not available. Scene detection functionality will be limited.


/usr/local/lib/python3.12/dist-packages/pydub/utils.py:300: SyntaxWarning: invalid escape sequence '\('
  m = re.match('([su]([0-9]{1,2})p?) \(([0-9]{1,2}) bit\)$', token)
/usr/local/lib/python3.12/dist-packages/pydub/utils.py:301: SyntaxWarning: invalid escape sequence '\('
  m2 = re.match('([su]([0-9]{1,2})p?)( \(default\))?$', token)
/usr/local/lib/python3.12/dist-packages/pydub/utils.py:310: SyntaxWarning: invalid escape sequence '\('
  elif re.match('(flt)p?( \(default\))?$', token):
/usr/local/lib/python3.12/dist-packages/pydub/utils.py:314: SyntaxWarning: invalid escape sequence '\('
  elif re.match('(dbl)p?( \(default\))?$', token):


WhisperJAV version: 1.7.6rc0


In [8]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [20]:
import os
import ipywidgets as widgets
from IPython.display import display

# Path to your input folder
input_folder = "/content/drive/MyDrive/WhisperJAV/input"

# List all mp4 files
video_files = [f for f in os.listdir(input_folder) if f.lower().endswith(".mp4")]

# Create dropdown widget
video_dropdown = widgets.Dropdown(
    options=video_files,
    description='Select Video:',
    layout=widgets.Layout(width='600px')
)

display(video_dropdown)

Dropdown(description='Select Video:', layout=Layout(width='600px'), options=('IPZZ-402-audio.mp4', 'IPZZ-383-o…

In [21]:
video_file = os.path.join(input_folder, video_dropdown.value)
print("Selected video:", video_file)

Selected video: /content/drive/MyDrive/WhisperJAV/input/IPZZ-383-org-audio.mp4


In [ ]:
!whisperjav "$video_file" --output-dir /content/drive/MyDrive/WhisperJAV/output


